In [29]:
import requests
import os

In [30]:
from googleapiclient.discovery import build #access YouTube Data API
import pandas as pd
#import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from IPython.display import JSON

In [31]:
#api_key = os.getenv('youtube_api_key')
api_key = ''
base_url = 'https://www.googleapis.com/youtube/v3'

In [32]:
channel_ids = ['UCvA_SWl8Ti3hvbmJyrLFqZQ', 'UCBZPPYU_l99Mg6-GXm6xbMg', 'UCBhPZ6lnTfQZn0bvARil-Xg', 'UCdM2hN2cxe1Ux0Bcn4Hslpg', 'UCRRSCAz5VJQqJHAIPgFVCxw', 'UCP6LRxQqx1wMNqlggF5qa_A',
               'UCjXVvJqLERkvsGTB-TKLx6Q', 'UCuRGQqKKdPOSPB0XG5VNk5g', 'UC8NZHJl-gfDZxbBvmL0vCXQ', 'UCcEIuIasgNTRoxjk5lI_t6A', 'UC12i9YDDwHaAKTBcQKzSFVw', 'UCOEK9oW_VYDHbX3qwT4JMyQ',
               'UCdPgo0qkTA5-PKq3YwRk5Gw', 'UCkBwgeT9gHVzALlMvQKi_ug', 'UCWDxHAG4OSFX8JRWiZmQ83w', 'UCikr3kOessKQkIMr2S3Pm4w', 'UC_X93AYMCJ3FViwBRyP5LZg', 'UCFojdS6BpyQwOBeo0I2WNpQ',
               'UCA-YlSjCrIdvEw6ivKRJeJg', 'UCl1txXouaN3YU8zzNPJP_pA', 'UCCe1fW83L5lfTfUPS11Py1Q', 'UCEly_O1P_iUpOV-QjsDXccw', 'UCYKXCpJbGe3fTYILjpJgW2A', 'UCY_kDUpfoqcrUQExsdKTX_g',
               'UCNzd8n4UapaAtrhC062RxDQ', 'UCsmPEw8gG-tmNJn_UzKwcWA', 'UCYGgbbl6veQIN9Hn0RhVX6A', 'UCaTYJcqWO4Y9e_-2nsDlmYA', 'UCmBsqNZEs2KF7PfZvXLd69A', 'UCGKRuOHpEl12cYYlJUO8nvw',
               'UCGCQW42TYud4pt-ois0Hq7Q', 'UC7UzatNIfKhrw4wG8DiJkaQ', 'UC7xlWEZWav-khk4IjevJX_Q', 'UCgdVgtJQXxebSiSAzlhYczw', 'UC2O6vD7gjgZIXKVegD6qOHA', 'UCGTO34YEPTNMwM3ezPCUCIQ',
               'UCNB-IDvDxuuiX2ghjNft5AQ', 'UCibcaePXd9ZIz3k9lTQvVRQ', 'UC3DgpJvB_izWfN56ZKCVSzA', 'UCn9mRGNo0CYj7nE6MepnWOQ', 'UCcegJZ-0FZcby0gsI4LAnGg', 'UC-upv67O6BrZuKh1HRIq6zg',
               'UCux-vyrlxszhL7JKm870sdA', 'UCoJ0b8z-FS_tuissbov-UUA', 'UCsYg_Sejxwi0nXe2ztzCXlw', 'UCwN-NLEsOx7cCiBeGap9LSA', 'UCihe5asUQZ32dqD0mOBPXog', 'UC7dCFKu_W8-veV354axaPOg'
               #more channels
               ]

In [33]:
api_service_name = "youtube"
api_version = "v3"    

        # Get credentials and create an API client
youtube = build(
            api_service_name, api_version, developerKey=api_key)

In [34]:
def get_channel_details(youtube, channel_ids): #fetch details
    all_data = []

    request = youtube.channels().list(
            part="snippet,contentDetails,statistics",
            id=','.join(channel_ids)
        )
    response = request.execute()

    for i in range(len(response['items'])):
        data = dict(channelName = response['items'][i]['snippet']['title'],
                    subscribers = response['items'][i]['statistics']['subscriberCount'],
                    views = response['items'][i]['statistics']['viewCount'],
                    totalVideos = response['items'][i]['statistics']['videoCount'],
                    playlistId = response['items'][i]['contentDetails']['relatedPlaylists']['uploads']
                    
                    )
        all_data.append(data)
    
    return pd.DataFrame(all_data)

#JSON(response)

Our csv file containing basic channel details, Kenyan_youtube2.csv

In [35]:
channel_data = get_channel_details(youtube, channel_ids)
channel_data.to_csv('Kenyan_youtube2.csv', index = False)

ServerNotFoundError: Unable to find the server at youtube.googleapis.com

Here, I get video statistics for all channels, getting to a total of 10126.
The statistics include specific music videos for each channel id, ratings(likes), date published, views and comment counts and more.
These will be used to further analyze and provide more insights.

In [ ]:
def get_video_ids(youtube, playlist_id):
    video_ids = []

    request = youtube.playlistItems().list(
        part = 'contentDetails',
        playlistId =playlist_id,
        maxResults = 50
    )
    response = request.execute()


    for i in range (len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')

    #return pd.DataFrame(video_ids)
        
    return video_ids


In [ ]:
def get_video_details(youtube, video_ids):
    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
            
    return pd.DataFrame(all_video_info)

In [ ]:
def fetch_youtube_data(youtube, channel_ids):
    
    # Fetch channel details
    channel_data = get_channel_details(youtube, channel_ids)
    
    # Initialize a list to collect video details DataFrames
    all_video_info = []

    # Loop through each channel to get video details
    for idx, row in channel_data.iterrows():
        playlist_id = row['playlistId']
        video_ids = get_video_ids(youtube, playlist_id)
        video_info_data = get_video_details(youtube, video_ids)
        all_video_info.append(video_info_data)

    
    video_info_data_combined = pd.concat(all_video_info, ignore_index=True)

    # Merge channel data with combined video details
    final_data = pd.merge(video_info_data_combined, channel_data, left_on='channelTitle', right_on='channelName', how='left')


    print(final_data)
    final_data.to_csv('kenyan_youtube_data1.csv', index=False)

    # Return the final DataFrame for further use if needed
    return final_data


In [ ]:
df = fetch_youtube_data(youtube, channel_ids)

ServerNotFoundError: Unable to find the server at youtube.googleapis.com

In [ ]:
def main(youtube, channel_ids):
    
    channel_df = get_channel_details(youtube, channel_ids)

    all_video_details = []

    for idx, row in channel_df.iterrows():
        playlist_id = row['playlistId']
        video_ids = get_video_ids(youtube, playlist_id)
        video_details = get_video_details(youtube, video_ids)
        all_video_details.append(video_details)

    all_videos_df = pd.concat(all_video_details, ignore_index=True)
    final_df = pd.merge(all_videos_df, channel_df, left_on='channelTitle', right_on='channelName', how='left')
    print(final_df)
    final_df.to_csv('kenyan_youtube_data.csv', index=False)

In [ ]:
df = main(youtube, channel_ids)
print(df)

          video_id  channelTitle  \
0      bMECgLgBvEg  Prince Indah   
1      c7gdO8V4VDE  Prince Indah   
2      _NqNg4iSE2M  Prince Indah   
3      rOeMGVqtu6I  Prince Indah   
4      sBdpjpgRWLw  Prince Indah   
...            ...           ...   
10121  fu_bOwPPkPg    Wanavokali   
10122  JWE5my57adM    Wanavokali   
10123  PURnxA02pKE    Wanavokali   
10124  tODe83D18vo    Wanavokali   
10125  wfc9QqGzobY    Wanavokali   

                                                   title  \
0           Prince Indah - Nyar Jaduong (Official Video)   
1         Prince Indah - Nyar Jaduong (Official Trailer)   
2                   Prince Indah - Puonj Mag Dak Jukebox   
3      MALAIKA MUSICALS FESTIVAL AT JOMO KENYATTA STA...   
4      Prince Indah - Mummy Chulo (Official Lyric Video)   
...                                                  ...   
10121                 2 in 1 - Naiboi (WANAVOKALI Cover)   
10122  Nviiri The Storyteller X WANAVOKALI - Pombe Si...   
10123                Namele